In [5]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [7]:
# ============================================================
# EEG Foundation Challenge 2025 - Challenge 1
# SuS pretrain → CCD RT fine-tune (NPZ-based version)
# ============================================================

import os, random, numpy as np, pandas as pd, warnings, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch.nn.functional as F
from glob import glob
import mne
import zipfile

warnings.filterwarnings("ignore")
mne.set_log_level("ERROR")

# ---------------- CONFIG ----------------
CACHE_DIR   = "/data5/open_data/HBN/cache_eeg_100hz_noref"
PRETRAIN_NPZ = os.path.join(CACHE_DIR, "pretrain_eeg.npz")
CCD_NPZ      = os.path.join(CACHE_DIR, "ccd_eeg.npz")
BIDS_ROOT   = "/data5/open_data/HBN/EEG_BIDS/"
TARGET_SFREQ = 100
WIN_S_SUS, WIN_S_CCD = 2.0, 2.0
STRIDE_S_SUS = 1.0
BATCH_SIZE, NUM_WORKERS = 64, 8
EPOCHS_SUS, EPOCHS_CCD, LR_SUS, LR_CCD = 5, 10, 1e-3, 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
random.seed(42); np.random.seed(42); torch.manual_seed(42); torch.cuda.manual_seed_all(42)

# ============================================================
# 1) NPZ 로드 함수
# ============================================================
def load_npz_dict(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"{path} not found. Please create it first.")
    print(f"[INFO] Loading NPZ archive: {path}")
    return np.load(path, allow_pickle=False)

# NPZ 두 개 로드
NPZ_PRE = load_npz_dict("/data5/open_data/HBN/cache_eeg_100hz_noref/pretrain_eeg.npz")
NPZ_CCD = load_npz_dict("/data5/open_data/HBN/cache_eeg_100hz_noref/ccd_eeg.npz")

def cached_load_from_npz(npz_dict, eeg_path):
    """NPZ 딕셔너리(npz_dict)에서 파일명 기반 key를 찾아 반환."""
    key = os.path.basename(eeg_path).replace("_cached.npy", "")
    # zipfile 구조는 key.npy로 저장되므로 두 가지 모두 시도
    if key in npz_dict:
        X = npz_dict[key]
    elif f"{key}.npy" in npz_dict:
        X = npz_dict[f"{key}.npy"]
    else:
        raise KeyError(f"❌ Missing key in NPZ: {key}")
    return X


def safe_get_from_npz(npz_dict, key):
    """NPZ 딕셔너리에서 key.npy를 안전하게 가져옴 (깨짐/중복 자동 skip)."""
    try:
        if key in npz_dict:
            return npz_dict[key]
        elif f"{key}.npy" in npz_dict:
            return npz_dict[f"{key}.npy"]
        else:
            # 중복 key는 마지막 entry만 남으므로 무시
            print(f"[WARN] Missing key in NPZ: {key} — skipped.")
            return None
    except (KeyError, OSError, ValueError, zipfile.BadZipFile) as e:
        print(f"[WARN] Corrupted entry for {key}: {e}")
        return None













# ============================================================
# 2) Helper
# ============================================================
def make_window(x_ct, center_s, sfreq=TARGET_SFREQ, win_sec=2.0):
    t1 = int(center_s * sfreq)
    Tw = int(win_sec * sfreq)
    t0 = max(0, t1 - Tw)
    seg = x_ct[:, t0:t1]
    need = Tw - seg.shape[1]
    if need > 0:
        seg = np.pad(seg, ((0, 0), (need, 0)), mode="constant")
    return seg.astype(np.float32)

def collect_ccd_event_files(bids_root):
    ev_files = glob(os.path.join(
        bids_root, "ds*/sub-*", "eeg", "sub-*_task-contrastChangeDetection*_events.tsv"
    ))
    print(f"✅ Found {len(ev_files)} CCD event files.")
    return ev_files

def match_cached_to_event(npz_keys, bids_root):
    ev_files = collect_ccd_event_files(bids_root)
    ev_dict = {os.path.basename(ef).replace("_events.tsv", ""): ef for ef in ev_files}
    pairs = []
    for key in npz_keys:
        base = key
        if base in ev_dict:
            pairs.append((base, ev_dict[base]))
        else:
            key_no_run = base.replace("_run-1", "").replace("_run-2", "")
            for k, v in ev_dict.items():
                if k.replace("_run-1", "").replace("_run-2", "") == key_no_run:
                    pairs.append((base, v)); break
    print(f"🔗 Matched {len(pairs)} EEG ↔ event pairs.")
    return pairs

def extract_ccd_trials(df):
    if df.empty or "onset" not in df.columns or "value" not in df.columns:
        return []
    trials = []
    on  = df["onset"].astype(float).values
    val = df["value"].astype(str).values
    fb  = df["feedback"].astype(str).values if "feedback" in df.columns else ["n/a"] * len(df)
    starts  = [i for i,v in enumerate(val) if "contrastTrial_start" in v]
    presses = [i for i,v in enumerate(val) if "buttonPress" in v]
    for ti in starts:
        t0 = on[ti]
        later = [pi for pi in presses if on[pi] > t0]
        if not later: 
            continue
        pi = later[0]
        rt = (on[pi]-t0) * 1000.0
        if 100 <= rt <= 3000 and "smiley" in fb[pi].lower():
            trials.append((t0, rt))
    return trials

# ============================================================
# 3) Datasets (NPZ 기반)
# ============================================================
class SusPretrainDataset(Dataset):
    def __init__(self, npz_dict, win_s=WIN_S_SUS, stride_s=STRIDE_S_SUS):
        self.keys = list(npz_dict.keys())
        self.items = []
        self.win_s = win_s; self.stride_s = stride_s
        for key in self.keys:
            X = npz_dict[key]
            T = X.shape[1]
            Tw = int(win_s * TARGET_SFREQ)
            stride = int(stride_s * TARGET_SFREQ)
            centers = [t1 / TARGET_SFREQ for t1 in range(Tw, T, stride)]
            if len(centers) == 0 and T >= Tw:
                centers = [Tw / TARGET_SFREQ]
            for c in centers:
                self.items.append((key, c))
        random.shuffle(self.items)

    def __len__(self): return len(self.items)

    def _augment(self, x):
        x = x + 0.01 * np.random.randn(*x.shape).astype(np.float32)
        if np.random.rand() < 0.5:
            L = max(1, int(x.shape[1]*0.1))
            s = np.random.randint(0, x.shape[1]-L+1)
            x[:, s:s+L] = 0.0
        if np.random.rand() < 0.5:
            drop = max(1, int(x.shape[0]*0.05))
            idx = np.random.choice(x.shape[0], drop, replace=False)
            x[idx] = 0.0
        return x



    def __getitem__(self, idx):
        key, c = self.items[idx]
        X = safe_get_from_npz(NPZ_PRE, key)
        if X is None:   # 파일 깨졌거나 중복된 경우 → 건너뛰기
            return None
        seg = make_window(X, c, win_sec=self.win_s)
        v1, v2 = self._augment(seg.copy()), self._augment(seg.copy())
        return torch.from_numpy(v1), torch.from_numpy(v2), torch.zeros(1)


class CcdRtDataset(Dataset):
    def __init__(self, npz_dict, eeg_event_pairs, win_s=WIN_S_CCD):
        self.samples = []
        for key, ev_path in eeg_event_pairs:
            if not os.path.exists(ev_path):
                continue
            df = pd.read_csv(ev_path, sep="\t")
            for o, rt in extract_ccd_trials(df):
                self.samples.append((key, o, rt))
        all_rts = np.array([s[2] for s in self.samples]).astype(np.float32)
        self.rt_mean, self.rt_std = all_rts.mean(), all_rts.std() + 1e-6
        print(f"✅ CCD Dataset: {len(self.samples)} trials. Mean RT={self.rt_mean:.2f}, Std={self.rt_std:.2f}")
        self.npz_dict, self.win_s = npz_dict, win_s

    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        key, o, rt = self.samples[idx]
        X = NPZ_CCD[key]
        seg = make_window(X, o, win_sec=self.win_s)
        rt_norm = (rt - self.rt_mean) / self.rt_std
        return torch.from_numpy(seg), torch.tensor([rt_norm], dtype=torch.float32)

# ============================================================
# 4) Model / Loss / Train loops
# ============================================================
from braindecode.models import EEGConformer

class SafeEEGConformerEncoder(nn.Module):
    def __init__(self, n_chans, sfreq, input_window_samples):
        super().__init__()
        for kw in [
            dict(n_chans=n_chans, n_outputs=1, n_times=input_window_samples, sfreq=sfreq, return_features=True),
            dict(n_chans=n_chans, n_outputs=1, input_window_samples=input_window_samples, sfreq=sfreq, return_features=True),
            dict(n_chans=n_chans, n_outputs=1, n_times=input_window_samples, sfreq=sfreq)
        ]:
            try:
                self.backbone = EEGConformer(**kw); break
            except TypeError: continue

    def forward(self, x):
        z = self.backbone(x)
        if isinstance(z, tuple): z = z[0]
        return torch.flatten(z, 1)

class RtHead(nn.Module):
    def __init__(self, feat_dim):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(feat_dim, max(64, feat_dim//2)),
            nn.ReLU(),
            nn.Linear(max(64, feat_dim//2), 1)
        )
    def forward(self, z):
        if z.ndim > 2: z = torch.flatten(z, 1)
        return self.mlp(z)

def nt_xent_loss(z1, z2, temperature=0.5):
    z1, z2 = F.normalize(z1, dim=1), F.normalize(z2, dim=1)
    logits12, logits21 = torch.matmul(z1, z2.T)/temperature, torch.matmul(z2, z1.T)/temperature
    logits12 -= logits12.max(dim=1, keepdim=True).values
    logits21 -= logits21.max(dim=1, keepdim=True).values
    labels = torch.arange(z1.size(0), device=z1.device)
    return (F.cross_entropy(logits12, labels) + F.cross_entropy(logits21, labels))/2

def train_pretrain_sus(dl, encoder, epochs=EPOCHS_SUS, lr=LR_SUS):
    with torch.no_grad():
        dummy, _, _ = next(iter(dl))
        feat_dim = encoder(dummy[:1].float().to(DEVICE)).shape[1]
    proj_head = nn.Sequential(
        nn.Linear(feat_dim, feat_dim // 2), nn.ReLU(), nn.Linear(feat_dim // 2, 128)
    ).to(DEVICE)
    opt = torch.optim.Adam(list(encoder.parameters()) + list(proj_head.parameters()), lr=lr)
    for ep in range(epochs):
        encoder.train(); proj_head.train(); losses=[]
        for x1, x2, _ in tqdm(dl, desc=f"[SuS pretrain] {ep+1}/{epochs}"):
            x1, x2 = x1.float().to(DEVICE), x2.float().to(DEVICE)
            z1, z2 = encoder(x1), encoder(x2)
            p1, p2 = proj_head(z1), proj_head(z2)
            loss = nt_xent_loss(p1, p2, temperature=2.0)
            opt.zero_grad(); loss.backward(); opt.step()
            losses.append(loss.item())
        print(f"Epoch {ep+1}: contrastive loss={np.mean(losses):.4f}")
    return encoder

def train_ccd_rt(dl_tr, encoder, rt_head, epochs=EPOCHS_CCD, lr=LR_CCD):
    opt = torch.optim.Adam(list(encoder.parameters()) + list(rt_head.parameters()), lr=lr)
    for ep in range(epochs):
        encoder.train(); rt_head.train(); losses=[]
        for x, y in tqdm(dl_tr, desc=f"[CCD train] {ep+1}/{epochs}"):
            x, y = x.float().to(DEVICE), y.to(DEVICE)
            yhat = rt_head(encoder(x))
            loss = F.l1_loss(yhat, y)
            opt.zero_grad(); loss.backward(); opt.step()
            losses.append(loss.item())
        print(f"Epoch {ep+1}: MAE={np.mean(losses):.3f}")


def collate_skip_none(batch):
    batch = [b for b in batch if b is not None]
    return torch.utils.data.dataloader.default_collate(batch)
# ============================================================
# 5) Main
# ============================================================
def main():
    # ---------- Pretrain ----------
    ds_sus = SusPretrainDataset(NPZ_PRE)
    dl_sus = DataLoader(ds_sus,batch_size=BATCH_SIZE,shuffle=True,num_workers=NUM_WORKERS,pin_memory=True,prefetch_factor=2,collate_fn=collate_skip_none)
    
    x_demo, _, _ = next(iter(dl_sus))
    _, C, T = x_demo.shape
    encoder = SafeEEGConformerEncoder(C, TARGET_SFREQ, T).to(DEVICE)
    encoder = train_pretrain_sus(dl_sus, encoder)
    torch.save(encoder.state_dict(), "encoder_sus_pretrained_npz.pth")
    print("✅ Saved encoder_sus_pretrained_npz.pth")

    # ---------- Fine-tune ----------
    keys_ccd = list(NPZ_CCD.keys())
    matched_pairs = match_cached_to_event(keys_ccd, BIDS_ROOT)
    ds_ccd = CcdRtDataset(NPZ_CCD, matched_pairs)
    dl_ccd = DataLoader(ds_ccd, batch_size=BATCH_SIZE, shuffle=True,
                        num_workers=NUM_WORKERS, pin_memory=True, prefetch_factor=2)
    with torch.no_grad():
        feat_dim = encoder(x_demo[:1].float().to(DEVICE)).shape[1]
    rt_head = RtHead(feat_dim).to(DEVICE)
    train_ccd_rt(dl_ccd, encoder, rt_head)
    torch.save({"encoder": encoder.state_dict(),
                "rt_head": rt_head.state_dict()}, "weights_ch1_npz.pth")
    print("✅ Saved Challenge-1 weights to weights_ch1_npz.pth")

if __name__ == "__main__":
    main()


[INFO] Loading NPZ archive: /data5/open_data/HBN/cache_eeg_100hz_noref/pretrain_eeg.npz
[INFO] Loading NPZ archive: /data5/open_data/HBN/cache_eeg_100hz_noref/ccd_eeg.npz
[WARN] Corrupted entry for sub-NDARBN611GRA_task-surroundSupp_run-1_eeg_pp: Overlapped entries: 'sub-NDARBN611GRA_task-surroundSupp_run-1_eeg_pp.npy' (possible zip bomb)[WARN] Corrupted entry for sub-NDARHC357HLP_task-RestingState_eeg_pp: Bad magic number for file header[WARN] Corrupted entry for sub-NDARBE091BGD_task-ThePresent_eeg_pp: File name in directory 'sub-NDARBE091BGD_task-ThePresent_eeg_pp.npy' and header b'sub-NDARLF616PBU_task-ThePresent_eeg_pp.npy' differ.[WARN] Corrupted entry for sub-NDARMP745EAC_task-FunwithFractals_eeg_pp: Bad magic number for file header[WARN] Corrupted entry for sub-NDARZC058BA5_task-RestingState_eeg_pp: Bad magic number for file header


[WARN] Corrupted entry for sub-NDARPL651MD5_task-ThePresent_eeg_pp: File name in directory 'sub-NDARPL651MD5_task-ThePresent_eeg_pp.npy' and heade

error: Caught error in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/RA/.local/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/home/RA/.local/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/RA/.local/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/tmp/ipykernel_3724404/1777488183.py", line 177, in __getitem__
    X = safe_get_from_npz(NPZ_PRE, key)
  File "/tmp/ipykernel_3724404/1777488183.py", line 60, in safe_get_from_npz
    return npz_dict[key]
  File "/home/RA/.local/lib/python3.10/site-packages/numpy/lib/npyio.py", line 256, in __getitem__
    return format.read_array(bytes,
  File "/home/RA/.local/l

[WARN] Corrupted entry for sub-NDARHC426PAW_task-DiaryOfAWimpyKid_eeg_pp: Overlapped entries: 'sub-NDARHC426PAW_task-DiaryOfAWimpyKid_eeg_pp.npy' (possible zip bomb)
[WARN] Corrupted entry for sub-NDARWU018VDB_task-surroundSupp_run-1_eeg_pp: Bad magic number for file header[WARN] Corrupted entry for sub-NDARPL596YTD_task-DiaryOfAWimpyKid_eeg_pp: File name in directory 'sub-NDARPL596YTD_task-DiaryOfAWimpyKid_eeg_pp.npy' and header b'sub-NDARGM421YWQ_task-DiaryOfAWimpyKid_eeg_pp.npy' differ.[WARN] Corrupted entry for sub-NDARGM421YWQ_task-DiaryOfAWimpyKid_eeg_pp: File name in directory 'sub-NDARGM421YWQ_task-DiaryOfAWimpyKid_eeg_pp.npy' and header b'sub-NDARWU018VDB_task-surroundSupp_run-1_eeg_pp.npy' differ.

[WARN] Corrupted entry for sub-NDARNX050XBN_task-surroundSupp_run-2_eeg_pp: File name in directory 'sub-NDARNX050XBN_task-surroundSupp_run-2_eeg_pp.npy' and header b'sub-NDARLE417RA4_task-seqLearning8target_eeg_pp.npy' differ.[WARN] Corrupted entry for sub-NDARLE417RA4_task-seqLearning8target_eeg_pp: Bad magic number for file header[WARN] Corrupted entry for sub-NDARZW980YF9_task-surroundSupp_run-2_eeg_pp: File name in directory 'sub-NDARZW980YF9_task-surroundSupp_run-2_eeg_pp.npy' and header b'sub-NDARNX050XBN_task-surroundSupp_run-2_eeg_pp.npy' differ.





[WARN] Corrupted entry for sub-NDAREB662RLE_task-DiaryOfAWimpyKid_eeg_pp: Overlapped entries: 'sub-NDAREB662RLE_task-DiaryOfAWimpyKid_eeg_pp.npy' (possible zip bomb)
[WARN] Corrupted entry for sub-NDARGC124PAW_task-symbolSearch_eeg_pp: File name in directory 'sub-NDARGC124PAW_task-symbolSearch_eeg_pp.npy' and header b'sub-NDAREX531PXT_task-symbolSearch_eeg_pp.npy' differ.
[WARN] Corrupted entry for sub-NDARUX885LMW_task-surroundSu